## Classify the presence and correct placement of tubes on chest x-rays to save lives

Note: This notebook is still in progress.
Any suggestions are welcome.

***Please don't forget to upvote, if you find this helpful.***

## TODO

* Improve readibility
    * Introductory Write-ups
    * More comments
* Image preprocessing
    * Current architecture requires 3 channel inputs, need to fix it.
    * Image preprocessing to improve the clarity of images
    * More augmentation
    * Different image sizes
* Class balancing
    * Weighted Loss Functions
    * Oversampling
* Architecture tuning
* Ensembling

In [ ]:
# ! pip install -q efficientnet >> /dev/null

# Training

## Import modules

In [ ]:
import os
import re
import math
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
import tensorflow.keras.backend as K
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [ ]:
import sys
sys.path.append('../input/tfresnestregnetdetrgenet')

from models.model_factory import get_model

In [ ]:
import sys
# sys.path.append('../input/efficientnet-keras-source-code')
# from efficientnet as efn

## Set parameters

In [ ]:
DATA_PATH = '/kaggle/input/ranzcr-clip-catheter-line-classification'

MODEL_PATH = '/kaggle/working/models'

In [ ]:
DEVICE = 'TPU' # ['CPU' GPU' 'TPU']

ENABLE_MIXED_PRECISION = True # [True False]

In [ ]:
SEED = 42

FOLDS = 5 

IMG_SIZE = 224 #512

BATCH_SIZE = 64 # [8, 16, 32, 64, 128, 256, 512]

EPOCHS = 25

VERBOSE = 1 # [0: silent, 1: progress bar, 2: single line]

In [ ]:
NUM_TF_RECS = len(os.listdir(f'{DATA_PATH}/train_tfrecords'))

print(NUM_TF_RECS)

## Setup devices and settings

In [ ]:
# For kaggle tpus
from kaggle_datasets import KaggleDatasets
if DEVICE == 'TPU':
    DATA_PATH = KaggleDatasets().get_gcs_path(DATA_PATH.split('/')[-1])

In [ ]:
if DEVICE == 'CPU':

    strategy = tf.distribute.get_strategy()
    print('\nUsing Default Distribution Strategy  for CPU')


if DEVICE == 'GPU':

    gpu_accelerarors = tf.config.list_physical_devices('GPU')
        
    if len(gpu_accelerarors) > 1:
        strategy = tf.distribute.MirroredStrategy()
        print(f'Number of GPUs available: {len(gpu_accelerarors)}')
        print('\n Using Mirrored Distribution Strategy')
        
    else:
        strategy = tf.distribute.get_strategy()
        if len(gpu_accelerarors) == 1:
            print(f'Number of GPUs available: 1')
            print('\nUsing Default Distribution Strategy for GPU')
        else:
            print('ERROR: GPU not available')
            print('\nUsing Default Distribution Strategy  for CPU')
        
if DEVICE == 'TPU':

    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        tpu_accelerarors = tf.config.list_logical_devices('TPU')
        print(f'Number of TPU cores available: {len(tpu_accelerarors)}')
        print(f'\nUsing TPU Distribution Strategy')
        
    except:
        print('ERROR: TPU not available')
        print('\nUsing Default Distribution Strategy for CPU')
        strategy = tf.distribute.get_strategy()
        
        
if ENABLE_MIXED_PRECISION:
    
    print('\nMixed Precision enabled:')
    
    if DEVICE == 'GPU':
        policy = mixed_precision.Policy('mixed_float16')
        
    if DEVICE == 'TPU':
        policy = mixed_precision.Policy('mixed_bfloat16')
        
    mixed_precision.set_policy(policy)
    
    print('\t...Compute dtype: %s' % policy.compute_dtype)
    print('\t...Variable dtype: %s' % policy.variable_dtype)


REPLICAS = strategy.num_replicas_in_sync
print(f'\nREPLICAS: {REPLICAS}')

## Helper functions

In [ ]:
class Dataset:
    
    feature_description = {
        "StudyInstanceUID"           : tf.io.FixedLenFeature([], tf.string),
        "image"                      : tf.io.FixedLenFeature([], tf.string),
        "ETT - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "ETT - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "ETT - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Incompletely Imaged"  : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "Swan Ganz Catheter Present" : tf.io.FixedLenFeature([], tf.int64),
    }
    
    def __init__(self, image_size):
        self.image_size = image_size
        
    def parse_function(self, example_proto):
        example = tf.io.parse_single_example(example_proto, self.feature_description)
        image = tf.io.decode_image(example['image'], channels=3)
        label = [example['ETT - Abnormal'],
                 example['ETT - Borderline'],
                 example['ETT - Normal'],
                 example['NGT - Abnormal'],
                 example['NGT - Borderline'],
                 example['NGT - Incompletely Imaged'],
                 example['NGT - Normal'],
                 example['CVC - Abnormal'],
                 example['CVC - Borderline'],
                 example['CVC - Normal'],
                 example['Swan Ganz Catheter Present']]
        return image, label 
    
    def augment_function(self, image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_contrast(image, 0.8, 1.2)
        image = tf.image.random_brightness(image, 0.1)   
        return image, label 
    
    def process_function(self, image, label):
        image.set_shape([None, self.image_size, self.image_size, 3])
        label.set_shape([None, 11])
        image = tf.image.resize(image, [224, 244], 'bilinear')/255 #[self.image_size, self.image_size], 'bilinear')/255
        return image, label
            
    def generator(self, files, batch_size=1, repeat=False, augment=False, shuffle=True):
        AUTO = tf.data.experimental.AUTOTUNE
        ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
        if shuffle: 
            opt = tf.data.Options()
            opt.experimental_deterministic = False
            ds = ds.with_options(opt)
            ds = ds.shuffle(2000)
        ds = ds.map(self.parse_function, num_parallel_calls=AUTO)
        if repeat:
            ds = ds.repeat()
        if augment:
            ds = ds.map(self.augment_function, num_parallel_calls=AUTO)
        ds = ds.batch(batch_size)
        ds = ds.map(self.process_function, num_parallel_calls=AUTO)
        ds = ds.prefetch(AUTO)
        return ds

## EfficientNet

def create_model(name, input_shape, classes, output_bias=None):
    
    # Dictionary mapping name to model function
    
    EFFICIENT_NETS = {'B0': efn.EfficientNetB0, 
                      'B1': efn.EfficientNetB1, 
                      'B2': efn.EfficientNetB2, 
                      'B3': efn.EfficientNetB3, 
                      'B4': efn.EfficientNetB4, 
                      'B5': efn.EfficientNetB5, 
                      'B6': efn.EfficientNetB6,
                      'B7': efn.EfficientNetB7}
    
    # Output layer bias initialization
    
    if output_bias is None:
        output_bias = 'zeros'
    else:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    
    # Base model
    
    base_model = EFFICIENT_NETS[name](include_top=False, 
                                      weights='imagenet', 
                                      input_shape=input_shape)
    
    # Model
    
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(classes, bias_initializer=output_bias)(x)
    outputs = tf.keras.layers.Activation('sigmoid', dtype='float32')(x) # Supports mixed-precision training
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
        
    metrics = [
        tf.keras.metrics.AUC(name='auc')
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

## Callbacks

In [ ]:
if not os.path.exists(MODEL_PATH):
        os.makedirs(MODEL_PATH)

In [ ]:
def create_callbacks(model_save_path, fold, verbose=1):
    
    verbose = int(verbose>0)
    
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)
    
    cpk_path = f'{model_save_path}/model-f{fold}.h5'

    checkpoint = ModelCheckpoint(
        filepath=cpk_path,
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        save_weights_only=True,
        verbose=verbose
    )

    reducelr = ReduceLROnPlateau(
        monitor='val_auc',
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=10, 
        verbose=verbose
    )
    
    callbacks = [checkpoint, reducelr, earlystop] #
    
    return callbacks

## count_items for counting the remained steps

In [ ]:
def count_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

## Main Training Pipeline

***Note: Running for 1 Fold only for experimental purpose***

In [ ]:
folds_val_auc = [None] * FOLDS # Store the validation auc for each fold

skf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

print(f'Training...')

for fold, (train_idx, valid_idx) in enumerate(skf.split(np.arange(NUM_TF_RECS))):
    
    print(f'\n\n{"*"*100} \nFOLD: {fold+1}')
    
    # Input Pipeline ******************************************************
    
    train_files = tf.io.gfile.glob(f'{DATA_PATH}/train_tfrecords/{idx:02}*.tfrec' for idx in train_idx)
    valid_files = tf.io.gfile.glob(f'{DATA_PATH}/train_tfrecords/{idx:02}*.tfrec' for idx in valid_idx)
    
    ds = Dataset(IMG_SIZE)
    
    train_ds = ds.generator(train_files, 
                            BATCH_SIZE*REPLICAS, 
                            repeat=True, 
                            augment=True, 
                            shuffle=True)

    valid_ds = ds.generator(valid_files, 
                            BATCH_SIZE*REPLICAS,  
                            repeat=False, 
                            augment=False, 
                            shuffle=False)
    
    # Calculate the steps_per_epoch
    
    steps_per_epoch = count_items(train_files)//(BATCH_SIZE*REPLICAS) * 2
    
    
    # Build Model ******************************************************
    
    tf.keras.backend.clear_session()
        
    with strategy.scope():
        model = get_model(model_name='ResNest50',
                          input_shape=[224,244,3],
                          n_classes=11,
                          fc_activation='softmax',
                          active='relu', # relu or mish
                          verbose=True)
        # model.summary()
        model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics =[tf.keras.metrics.AUC(name='auc')])
    
    print(f'\nModel initialized and compiled: ResNest50')
    
#         model = create_model(name=EFF_NET, 
#                      input_shape=(IMG_SIZE,IMG_SIZE,3), 
#                      classes=11)
#         model = compile_model(model, lr=0.0001)
        
#     print(f'\nModel initialized and compiled: EfficientNet-{EFF_NET}')
    
    
        
    # Train ******************************************************
   
    callbacks = create_callbacks(MODEL_PATH, fold+1, verbose=VERBOSE)

    print(f'\nModel training...\n')
    
    history = model.fit(train_ds, 
                        epochs=EPOCHS, #EPOCHS, 
                        steps_per_epoch=steps_per_epoch,
                        validation_data=valid_ds,
                        callbacks=callbacks,
                        verbose=True)

#     break
#     # Save acc for each fold in a list
#     folds_val_auc[fold] = max(history.history['val_auc'])
    
#     print(f'\nModel trained \n\nFOLD-{fold+1} Validation AUC = {folds_val_auc[fold]}')
    
#     break

## Traing Graph

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history['auc'], 'r', label='train auc')
    plt.plot(hist.history['val_auc'], 'g', label='val auc')
    plt.title("model auc")
    plt.ylabel("auc")
    plt.xlabel("epoch")
    plt.legend(loc='upper left')
    plt.show()


plot_hist(history)

regnety400  
```
elif model_name == 'resnest_detr':
            model = ResNest50_DETR(verbose=verbose, input_shape=input_shape,
            n_classes=n_classes, dropout_rate=dropout_rate, fc_activation=fc_activation, **kwargs).build()
    elif model_name == 'genet_light':
            model = GENet(verbose=verbose, model_name='light',input_shape=input_shape,
            n_classes=n_classes, fc_activation=fc_activation, **kwargs).build()
    elif model_name == 'genet_normal':
            model = GENet(verbose=verbose, model_name='normal',input_shape=input_shape,
            n_classes=n_classes, fc_activation=fc_activation, **kwargs).build()
    elif model_name == 'genet_large':
            model = GENet(verbose=verbose, model_name='large',input_shape=input_shape,
            n_classes=n_classes, fc_activation=fc_activation, **kwargs).build()
```

# Inference
- (Run cells Above) Setup devices and settings

## Import modules

In [ ]:
import re,os,cv2,random

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import albumentations as A
from sklearn.model_selection import train_test_split

# importing neural network architecture
# from efficientnet.tfkeras import EfficientNetB7

# importing other useful tools: layers, optimizers, loss functions
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

%matplotlib inline 
print("Tensorflow version " + tf.__version__)

## Path Setting

In [ ]:
WORK_DIR = '../input/ranzcr-clip-catheter-line-classification'
os.listdir(WORK_DIR)

GCS_DS_PATH = KaggleDatasets().get_gcs_path('ranzcr-clip-catheter-line-classification')

In [ ]:
# data connection
train = pd.read_csv(os.path.join(WORK_DIR, "train.csv"))
# train.display(5)

train_images = GCS_DS_PATH + "/train/" + train['StudyInstanceUID'] + '.jpg'

ss = pd.read_csv(os.path.join(WORK_DIR, 'sample_submission.csv'))
test_images = GCS_DS_PATH + "/test/" + ss['StudyInstanceUID'] + '.jpg'

label_cols = ss.columns[1:]
labels = train[label_cols].values

# train_annot = pd.read_csv(os.path.join(WORK_DIR, "train_annotations.csv"))

In [ ]:
# main parameters
AUTO = tf.data.experimental.AUTOTUNE
# TARGET_SIZE = 224

In [ ]:
def build_decoder(with_labels = True,
                  target_size = (224, 244), 
                  ext = 'jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

# in this part you can choose any type of augmentation from the ones suggested above
def build_augmenter(with_labels = True):
    def augment(img):
        #img = NeedleAugmentation(img, n_needles=2, dark_needles=False, p=0.5)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img, 0.9, 1)
        img = tf.image.random_contrast(img, 0.9, 1)
        #img = tf.image.random_saturation(img, 0.9, 1) 
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = 32, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
test_df = build_dataset(
    test_images, bsize = BATCH_SIZE, repeat = False, 
    shuffle = False, augment = False, cache = False)


In [ ]:
%time
pred = model.predict(test_df, verbose=1)
pred.shape

In [ ]:
with strategy.scope():
    modelld = get_model(model_name='ResNest50',
                        input_shape=[224,244,3],
                        n_classes=11,
                        fc_activation='softmax',
                        active='relu', # relu or mish
                        verbose=True)
    modelld.load_weights('./models/model-f1.h5')
modelld.summary()

In [ ]:
maxpred = modelld.predict(test_df, verbose=1)
maxpred

## Submmit

In [ ]:
sm_df = pd.DataFrame(columns=train.columns.tolist()[:-1])
for i, p in enumerate(maxpred):
    sm_df = sm_df.append(pd.Series([ss['StudyInstanceUID'][i]] + list(map(float,p)), index=sm_df.columns), ignore_index=True)
sm_df

In [ ]:
tr_df = pd.DataFrame(columns=train.columns.tolist()[:-1])
for i, p in enumerate(pred):
    tr_df = tr_df.append(pd.Series([ss['StudyInstanceUID'][i]] + list(map(float,p)), index=tr_df.columns), ignore_index=True)
tr_df

In [ ]:
sm_df.to_csv('submission.csv', index = False)

In [ ]:
# tr_df.to_csv('submission.csv', index = False)

In [ ]:
# model.save_weights("resnest50_weights.h5")

In [ ]:
# model.save("resnest50.h5")

In [ ]:
# np.savetxt('test.csv',[1,2,3,4,5], fmt='%f', delimiter=',', header='StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal', comments='')

In [ ]:
# print('Generating submission.csv file...')
# # test_ids_ds = test_df.map(lambda image, idnum: idnum).unbatch()
# # test_ids = next(iter()).numpy().astype('U') # all in one batch

# np.savetxt('submission.csv', np.rec.fromarrays([ss['StudyInstanceUID'][i]] + [*pred[i]]), fmt=['%s', '%f','%f' , '%f', '%f','%f' , '%f', '%f','%f' , '%f', '%f','%f'  ], delimiter=',', header='StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present', comments='')

def activation_layer_vis(img, activation_layer = 0, layers = 10):
    layer_outputs = [layer.output for layer in model.layers[:layers]]
    activation_model = models.Model(inputs = model.input, outputs = layer_outputs)
    activations = activation_model.predict(img)
    
    rows = int(activations[activation_layer].shape[3] / 3)
    cols = int(activations[activation_layer].shape[3] / rows)
    fig, axes = plt.subplots(rows, cols, figsize = (15, 15 * cols))
    axes = axes.flatten()
    
    for i, ax in zip(range(activations[activation_layer].shape[3]), axes):
        ax.matshow(activations[activation_layer][0, :, :, i], cmap = 'viridis')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

activation_layer_vis(img_tensor)

img_tensor = build_dataset(
    pd.Series(train_img[0]), bsize = 1,repeat = False, 
    shuffle = False, augment = False, cache = False)

x = tf.keras.layers.Lambda(lambda x: tf.math.divide(x, tf.reduce_max(x)))(x)

embed_layer = tf.keras.layers.Lambda(lambda x: x + get_position_encoding(timesteps, embed_size))(embed_layer)